In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
input_fname = './forward_backward/test_IOTA.in'
np.set_printoptions(formatter={'float': lambda x: format(x, '1.3E')})

# Prepare Beam and Lattice

In [2]:
beam, lattice0 = impact.readInputFile(input_fname)
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=8
beam.current=0.0
beam.integrator = 'Linear'
beam.distribution.distribution_type = 'Waterbag'
beam.subcycle = True

reading ImpactZ input file (./forward_backward/test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


### subcycle = 1

In [3]:
quad = impact.getElem('quad')
quad.length = 0.1
quad.B1 = 15.0
quad.n_sckick = 1
quad.n_map = 1
latticeNoSub = [impact.getElem('loop_through_lattice'),
               impact.getElem('write_raw_ptcl'),
               quad,
               impact.getElem('write_raw_ptcl')]

latticeNoSub[1].file_id = 1001
latticeNoSub[3].file_id = 1002

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

impact.writeInputFile(beam,latticeNoSub);
impact.run()
pDataNoSub0 = impact.readParticleData(1001,ke,mass,freq)
pDataNoSub1 = impact.readParticleData(1002,ke,mass,freq)

input error <- sum(beam.multi_charge.n_particles) not qual to beam.n_particles
  ... enforcing  beam.multi_charge.n_particles[0] to beam.n_particles
input error <- beam.multi_charge.current[0] not qual to beam.current
  ... enforcing  beam.multi_charge.current[0] to beam.current


### subcycle = 10

In [4]:
latticeSub = copy(latticeNoSub)
latticeSub[2].n_map = 10

impact.writeInputFile(beam,latticeSub);
impact.run()
pDataSub0 = impact.readParticleData(1001,ke,mass,freq)
pDataSub1 = impact.readParticleData(1002,ke,mass,freq)

### test

In [5]:
print(pDataSub0[:,:6]-pDataNoSub0[:,:6])

[[0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]]


In [6]:
print(pDataSub1[:,:6]-pDataNoSub1[:,:6])

[[0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]
 [0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00]]


### check impact input file

In [7]:
impact.writeInputFile(beam,latticeSub,'testSub.in');
impact.writeInputFile(beam,latticeNoSub,'testNoSub.in');

In [8]:
%%bash
diff testSub.in testNoSub.in

16c16
< 0.1 1 10 1 15.0 0 1.0 0.0 0.0 0.0 0.0 0.0 / 
---
> 0.1 1 1 1 15.0 0 1.0 0.0 0.0 0.0 0.0 0.0 / 


In [9]:
cat testSub.in

!================= Beam & Control Parameters ================= 
1 1 
6 8 1 0 1 
64 64 1025 7 0.05 0.05 0.1 
3 0 1 1 
8 
0.0 
1.0657889726792521e-09 
0.0007108541000000002 7.481793000000001e-05 0.8153394 1.0 1.0 0.0 0.0 
0.0007108541000000002 7.481793000000001e-05 0.8153394 1.0 1.0 0.0 0.0 
2.2092772151898727 1e-14 0.0 1.0 1.0 0.0 0.0 
0.0 2500000.0 938272045.0 1.0 30000000.0 0.0 
!========================== Lattice ========================== 
0.0 0 0 -16 0.0 1 / 
0.0 0 1001 -2 / 
0.1 1 10 1 15.0 0 1.0 0.0 0.0 0.0 0.0 0.0 / 
0.0 0 1002 -2 / 
